In [ ]:
# Network Algorithms Homework Project

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# returns all the information about an airport by its IATA
def airport_info(data_airports, iata):
    return data_airports.loc[data_airports['IATA'] == iata]

# Raw DataFrame to be cleaned  
data_airports = pd.read_csv('airports.csv')
data_routes = pd.read_csv('routes.csv')

# to display all the columns
# pd.set_option('display.max_columns', None)

# for checking
print(data_airports.head())
print(data_routes.head())

# for checking
print(airport_info(data_airports,'GYD'))
print(airport_info(data_airports,'HEL'))
print(airport_info(data_airports,'NRT'))

# array dist is an array of all the distances between sources and destinations that should be created and filled by Rustam
# once it's filled a new data frame with the Source, Destination and Distance values will be created
dist = []
data = [data_routes['Source airport'],data_routes['Destination airport'],dist]

# new DataFrame only with the Source, Destination and Distance values
df = pd.DataFrame(data, columns = ['Source', 'Dest', 'Dist'])